In [0]:
import os
import numpy as np
import keras as k
import pandas as pd
import string
from nltk.corpus import stopwords
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Embedding, Convolution1D, GlobalMaxPool1D, Flatten, Dense, Dropout, LSTM
from keras.layers import Bidirectional, MaxPooling1D, Lambda, MaxPooling2D, Reshape
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.utils import to_categorical
from keras.backend import one_hot

# Dense, Dropout, Flatten, Input, MaxPooling1D, Convolution1D, Embedding
# from gensim.models import KeyedVectors
# from keras.utils import to_categorical
# from nltk.corpus import stopwords

Using TensorFlow backend.


### Load in the files

In [0]:
import pickle
def load_tokenizer(filename = 'tokenizer.pickle'):
    with open(filename, 'rb') as handle:
        tokenizer = pickle.load(handle)
    return tokenizer
def load_df(filename):
    return pd.read_pickle(filename)
def load_embedding_matrix(filename = 'embedding_matrix.npy'):
    return np.load(filename)

In [0]:
t = load_tokenizer()
embedding_matrix = load_embedding_matrix()
df_train = load_df('df_train.pkl')
df_val = load_df('df_val.pkl')
df_test = load_df('df_test.pkl')
df_train = df_train.replace('', 'nan')

l2i = {'pants-fire':0, 'false':1, 'barely-true':2, 'half-true':3, 'mostly-true':4, 'true':5}

In [0]:
def df_to_input(df, tokenizer, max_len):
    return pad_sequences(tokenizer.texts_to_sequences(df.p_statement), maxlen=max_len, padding='post', truncating='post')
def df_to_label(df):
    return to_categorical(df.label)

In [0]:
print(np.median(df_train['length']))

10.0


In [0]:
sentence_max_len = 15
X_train = df_to_input(df_train, t, sentence_max_len)
X_val = df_to_input(df_val, t, sentence_max_len)
X_test = df_to_input(df_test, t, sentence_max_len)

Y_train = df_to_label(df_train)
Y_val = df_to_label(df_val)
Y_test = df_to_label(df_test)

### Model

In [0]:
# Liar
sentence_max_len = X_train.shape[1]
filter_sizes = (2,3,4)
n_filters = 128
vocab_size = len(t.word_index) + 1
dimensions = 300
input_shape = (sentence_max_len,)
n_categories = len(l2i)
batch_size = 64
epochs = 10
dropout_prob = 0.8

# Github
sentence_max_len = X_train.shape[1]
filter_sizes = (3,3,3)
n_filters = 128
vocab_size = len(t.word_index) + 1
dimensions = 300
input_shape = (sentence_max_len,)
n_categories = len(l2i)
batch_size = 40
epochs = 30
dropout_prob = 0.6

In [0]:
# Input layer
input_layer = Input(shape=input_shape)

# Embedding layer
embedding_layer = Embedding(vocab_size, dimensions, weights=[embedding_matrix], 
              input_length=sentence_max_len,trainable=False, name='embedding')(input_layer)

convolutional_blocks = []
for i, filter_size in enumerate(filter_sizes):
    convolution_layer = Convolution1D(filters=n_filters, kernel_size=filter_size, 
                                      padding="valid", activation="relu", strides = 1,
                                      name='conv_layer' + str(i) + "_" + str(filter_size) + "_" 
                                      + str(n_filters))(embedding_layer)
    convolution_layer = GlobalMaxPool1D()(convolution_layer)
    convolutional_blocks.append(convolution_layer)
if (len(filter_sizes)>1):
    conv = Concatenate()(convolutional_blocks)
else:
    # untested
    conv = convolution_layer

dropout_layer = Dropout(dropout_prob, name="dropout")(conv)
dense_layer = Dense(np.sum(n_filters), activation='relu', name='dense')(dropout_layer)
output_layer = Dense(n_categories, activation='softmax', name='output_dense')(dense_layer)

model = Model(input_layer, output_layer)
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=["categorical_accuracy"])

In [0]:
model.fit(X_train, Y_train, batch_size = batch_size, epochs=epochs, validation_data=(X_val, Y_val))

Train on 10240 samples, validate on 1284 samples
Epoch 1/30
10240/10240 [==============================] - 7s 654us/step - loss: 1.7632 - categorical_accuracy: 0.2146 - val_loss: 1.7355 - val_categorical_accuracy: 0.2220
Epoch 2/30
10240/10240 [==============================] - 6s 615us/step - loss: 1.7172 - categorical_accuracy: 0.2493 - val_loss: 1.7212 - val_categorical_accuracy: 0.2181
Epoch 3/30
10240/10240 [==============================] - 6s 621us/step - loss: 1.6785 - categorical_accuracy: 0.2806 - val_loss: 1.7108 - val_categorical_accuracy: 0.2438
Epoch 4/30
10240/10240 [==============================] - 6s 623us/step - loss: 1.6281 - categorical_accuracy: 0.3149 - val_loss: 1.7031 - val_categorical_accuracy: 0.2259
Epoch 5/30
10240/10240 [==============================] - 6s 633us/step - loss: 1.5682 - categorical_accuracy: 0.3530 - val_loss: 1.7138 - val_categorical_accuracy: 0.2632
Epoch 6/30
10240/10240 [==============================] - 6s 635us/step - loss: 1.4805 - ca

In [0]:
# model.predict(X_test, batch_size = batch_size)

In [0]:
prediction = model.predict(X_test, batch_size = batch_size)
Y_labels_classes = df_test.label.to_numpy()
np.sum(np.argmax(prediction, axis=1) == Y_labels_classes)/Y_labels_classes.shape[0]

0.2430939226519337

### Hybrid CNN

In [0]:
def load_dep_meta_pos(path = "./features/dep_meta_pos/", feature = "POS" ):
    train = np.load(path + "Train_" + feature +".npy")
    val = np.load(path + "Valid_" + feature +".npy")
    test = np.load(path + "Test_" + feature +".npy")
    return train, val, test
def load_pos():
    return load_dep_meta_pos(feature="POS")
def load_dep():
    return load_dep_meta_pos(feature="DEP")
def load_meta():
    return load_dep_meta_pos(feature="Meta")
senti2int = {"textblob":1, "vader":2, "lr": 3, "svm": 4, "ft":5, "flair_glove": 6, "flair_elmo": 7, "flair_bert": 8}
def load_senti(path = "./features/senti/", feature = "flair_glove" ):
    if isinstance(feature, str):
        feature = senti2int[feature.lower()]
    feature = str(feature)
    train = np.load(path + "Train_Senti_" + feature +".npy")
    train_class = np.load(path + "Train_SentiClass_" + feature +".npy")

    val = np.load(path + "Valid_Senti_" + feature +".npy")
    val_class = np.load(path + "Valid_SentiClass_" + feature +".npy")
    
    test = np.load(path + "Test_Senti_" + feature +".npy")
    test_class = np.load(path + "Test_SentiClass_" + feature +".npy")
    return train, train_class, val, val_class, test, test_class

In [0]:
import pickle
def load_tokenizer(filename='tokenizer.pickle'):
    with open(filename, 'rb') as handle:
        tokenizer = pickle.load(handle)
    return tokenizer
def load_tokenizers(count=6, filename='tokenizer'):
    ts = []
    for i in range(count):
        ts.append(load_tokenizer(filename=filename+"_"+str(i)+".pickle"))
    return ts
cols = ['subject', 'speaker', 'job', 'state', 'party', 'venue']

In [0]:
ts = load_tokenizers()

In [0]:
def df_to_meta_input(df, tokenizers, columns):
    a = np.zeros((df.shape[0], len(columns)), dtype=int)
    for i, col in enumerate(columns):
        a[:,i] = np.array(tokenizers[i].texts_to_sequences(df[col])).reshape(-1)
    return a

In [0]:
X_meta_train = df_to_meta_input(df_train, ts, cols)
X_meta_val = df_to_meta_input(df_val, ts, cols)
X_meta_test = df_to_meta_input(df_test, ts, cols)

In [0]:
train_pos, val_pos, test_pos = load_pos()
train_dep, val_dep, test_dep = load_dep()
train_meta, val_meta, test_meta = load_meta()

train_senti, train_senti_class, val_senti, val_senti_class, test_senti, test_senti_class = load_senti(feature = 8)

In [0]:
# Liar
sentence_max_len = X_train.shape[1]
filter_sizes = (2,3,4)
n_filters = 128
vocab_size = len(t.word_index) + 1
dimensions = 300
we_input_shape = (sentence_max_len,)
n_categories = len(l2i)
batch_size = 64
epochs = 5
dropout_prob = 0.8

number_of_meta_features = X_meta_train.shape[1]
md_input_shape = (number_of_meta_features,)
md_dimensions = dimensions
# number_of_meta_features = 10
md_embedding_input_size = 2
md_filter_sizes = (3,6)
md_n_filters = 10
lstm_size = 3
max_pooling_pool_size = 1

In [0]:
################################## Word Embeddings ~ we ##################################
# Input layer
we_input = Input(shape=we_input_shape, name='we_input')

# Embedding layer
we_embedding_layer = Embedding(vocab_size, dimensions, weights=[embedding_matrix], 
              input_length=sentence_max_len,trainable=False, name='we_embedding')(we_input)

convolutional_blocks = []
for i, filter_size in enumerate(filter_sizes):
    convolution_layer = Convolution1D(filters=n_filters, kernel_size=filter_size, 
                                      padding="valid", activation="relu", strides = 1,
                                      name='conv_layer' + str(i) + "_" + str(filter_size) + "_" 
                                      + str(n_filters))(we_embedding_layer)
    convolution_layer = GlobalMaxPool1D()(convolution_layer)
    convolutional_blocks.append(convolution_layer)
# print(we_input)
# if (len(filter_sizes)>1):
#     conv = Concatenate()(convolutional_blocks)
# else:
#     # untested
#     conv = convolution_layer

################################## Meta Data ##################################
# Input layer
# md_input = Input(shape=md_input_shape, name='md_input')

### Embedding layers ###
# md_input_split = Lambda(lambda x: tf.split(x, number_of_meta_features, axis=1))(md_input)
# print(md_input_split)
all_inputs = []
all_inputs.append(we_input)

md_input = Input(shape=md_input_shape, name="md_input")
all_inputs.append(md_input)
md_input_split = Lambda(lambda x: tf.split(x, number_of_meta_features, axis=1))(md_input)

meta_embedding_layers = []
for i, c in enumerate(cols):
#     md_input = Input(shape=(1,))
    vocab_dim = len(ts[i].word_index) + 1
#     md_inputs.append(md_input)
    meta_embedding_layer = Embedding(vocab_dim, md_dimensions)(md_input_split[i])
    meta_embedding_layers.append(meta_embedding_layer)
md_embed_concat = Concatenate(axis=1, name='md_embed_concat')(meta_embedding_layers)

# md_embedding_layer = Embedding(md_embedding_input_size, dimensions, input_length=number_of_meta_features, 
#                                name='md_embedding_layer')(md_input)
# print(md_embed_concat)
# print(we_embedding_layer)
md_convolutional_blocks = []
for i, filter_size in enumerate(md_filter_sizes):
    md_convolution_layer = Convolution1D(filters=md_n_filters, kernel_size=filter_size, 
                                      padding="valid", activation="relu", strides = 1,
                                      name='conv_layer' + str(i) + "_" + str(filter_size) + "_" 
                                      + str(n_filters))(md_embed_concat)
#     print(filter_size)
#     md_convolution_layer = MaxPooling1D(pool_size=max_pooling_pool_size)(md_convolution_layer)
    md_convolution_layer = GlobalMaxPool1D()(md_convolution_layer)

    md_convolutional_blocks.append(md_convolution_layer)
if (len(filter_sizes)>1):
    md_conv = Concatenate()(md_convolutional_blocks)
else:
    # untested
    md_conv = m_d_convolution_layer

reshape_md_conv = Reshape((md_conv.shape[1], 1))(md_conv)

md_bilstm = Bidirectional(LSTM(lstm_size, return_sequences=False))(reshape_md_conv)

################################## Concat ##################################
convolutional_blocks.append(md_bilstm)

if (len(filter_sizes)>1):
    conv = Concatenate()(convolutional_blocks)
else:
    # untested
    conv = convolution_layer

################################## Output ##################################
dropout_layer = Dropout(dropout_prob, name="dropout")(conv)

dense_layer = Dense(n_filters, activation='relu', name='dense')(dropout_layer)
output_layer = Dense(n_categories, activation='softmax', name='output_dense')(dense_layer)

model = Model(all_inputs, output_layer)
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=["categorical_accuracy"])

In [0]:
# model.summary()

In [0]:
model.fit([X_train, X_meta_train], Y_train, batch_size = batch_size, epochs=epochs, validation_data=([X_val, X_meta_val], Y_val))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 10240 samples, validate on 1284 samples
Epoch 1/5
10240/10240 [==============================] - 16s 2ms/step - loss: 1.8029 - categorical_accuracy: 0.1977 - val_loss: 1.7556 - val_categorical_accuracy: 0.2126
Epoch 2/5
10240/10240 [==============================] - 15s 1ms/step - loss: 1.7426 - categorical_accuracy: 0.2303 - val_loss: 1.7386 - val_categorical_accuracy: 0.2399
Epoch 3/5
10240/10240 [==============================] - 14s 1ms/step - loss: 1.7253 - categorical_accuracy: 0.2442 - val_loss: 1.7156 - val_categorical_accuracy: 0.2687
Epoch 4/5
10240/10240 [==============================] - 14s 1ms/step - loss: 1.6944 - categorical_accuracy: 0.2636 - val_loss: 1.6847 - val_categorical_accuracy: 0.2671
Epoch 5/5
10240/10240 [==============================] - 15s 1ms/step - loss: 1.6241 - categorical_accuracy: 0.3089 - val_loss: 1.6847 - val_categorical_accuracy: 0.2523


In [0]:
prediction = model.predict([X_test, X_meta_test], batch_size = batch_size)
Y_labels_classes = df_test.label.to_numpy()
np.sum(np.argmax(prediction, axis=1) == Y_labels_classes)/Y_labels_classes.shape[0]

0.24861878453038674

# Tuning - Hybrid CNN

In [0]:
#!pip install talos  #dependancy
import talos

sentence_max_len = X_train.shape[1]
filter_sizes = (2,3,4)
#n_filters = 128
vocab_size = len(t.word_index) + 1
dimensions = 300
we_input_shape = (sentence_max_len,)
n_categories = len(l2i)
#batch_size = 64
#epochs = 5
dropout_prob = 0.8

number_of_meta_features = X_meta_train.shape[1]
md_input_shape = (number_of_meta_features,)
md_dimensions = dimensions
# number_of_meta_features = 10
md_embedding_input_size = 2
md_filter_sizes = (3,6)
md_n_filters = 10
lstm_size = 3
max_pooling_pool_size = 1

def fakeNewsModel(x_train, y_train, x_val, y_val, params):
  we_input = Input(shape=we_input_shape, name='we_input')

  # Embedding layer
  we_embedding_layer = Embedding(vocab_size, dimensions, weights=[embedding_matrix], 
              input_length=sentence_max_len,trainable=False, name='we_embedding')(we_input)

  convolutional_blocks = []
  for i, filter_size in enumerate(filter_sizes):
      convolution_layer = Convolution1D(filters=params['n_filters'], kernel_size=filter_size, 
                                        padding="valid", activation="relu", strides = 1,
                                        name='conv_layer' + str(i) + "_" + str(filter_size) + "_" 
                                        + str(params['n_filters']))(we_embedding_layer)
      convolution_layer = GlobalMaxPool1D()(convolution_layer)
      convolutional_blocks.append(convolution_layer)

  all_inputs = []
  all_inputs.append(we_input)

  md_input = Input(shape=md_input_shape, name="md_input")
  all_inputs.append(md_input)
  md_input_split = Lambda(lambda x: tf.split(x, number_of_meta_features, axis=1))(md_input)

  meta_embedding_layers = []
  for i, c in enumerate(cols):
#     md_input = Input(shape=(1,))
      vocab_dim = len(ts[i].word_index) + 1
#     md_inputs.append(md_input)
      meta_embedding_layer = Embedding(vocab_dim, md_dimensions)(md_input_split[i])
      meta_embedding_layers.append(meta_embedding_layer)
  md_embed_concat = Concatenate(axis=1, name='md_embed_concat')(meta_embedding_layers)

  md_convolutional_blocks = []
  for i, filter_size in enumerate(md_filter_sizes):
      md_convolution_layer = Convolution1D(filters=md_n_filters, kernel_size=filter_size, 
                                        padding="valid", activation="relu", strides = 1,
                                        name='conv_layer' + str(i) + "_" + str(filter_size) + "_" 
                                        + str(n_filters))(md_embed_concat)
#     print(filter_size)
#     md_convolution_layer = MaxPooling1D(pool_size=max_pooling_pool_size)(md_convolution_layer)
      md_convolution_layer = GlobalMaxPool1D()(md_convolution_layer)

      md_convolutional_blocks.append(md_convolution_layer)
  if (len(filter_sizes)>1):
      md_conv = Concatenate()(md_convolutional_blocks)
  else:
    # untested
      md_conv = m_d_convolution_layer

  reshape_md_conv = Reshape((md_conv.shape[1], 1))(md_conv)

  md_bilstm = Bidirectional(LSTM(lstm_size, return_sequences=False))(reshape_md_conv)

################################## Concat ##################################
  convolutional_blocks.append(md_bilstm)

  if (len(filter_sizes)>1):
      conv = Concatenate()(convolutional_blocks)
  else:
    # untested
      conv = convolution_layer

################################## Output ##################################
  dropout_layer = Dropout(dropout_prob, name="dropout")(conv)

  dense_layer = Dense(n_filters, activation='relu', name='dense')(dropout_layer)
  output_layer = Dense(n_categories, activation='softmax', name='output_dense')(dense_layer)

  model = Model(all_inputs, output_layer)
  model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=["categorical_accuracy"])
  out = model.fit([X_train, X_meta_train], Y_train, batch_size = params['batch_size'], epochs=params['epochs'], validation_data=([X_val, X_meta_val], Y_val))
  return out, model


p = {'n_filters':[32, 64, 128],
      'batch_size': [10, 50, 64, 100],
      'epochs': [5, 10, 20]}

scan_object = talos.Scan(x=[X_train, X_meta_train],
                         y=Y_train,
                         x_val=[X_val, X_meta_val],
                         y_val=Y_val,
                         params=p,
                         model= fakeNewsModel,
                         experiment_name="test")


  0%|          | 0/36 [00:00<?, ?it/s]

Train on 10240 samples, validate on 1284 samples
Epoch 1/5
10240/10240 [==============================] - 67s 7ms/step - loss: 1.7759 - categorical_accuracy: 0.2056 - val_loss: 1.7366 - val_categorical_accuracy: 0.2321
Epoch 2/5
10240/10240 [==============================] - 66s 6ms/step - loss: 1.7351 - categorical_accuracy: 0.2358 - val_loss: 1.7209 - val_categorical_accuracy: 0.2399
Epoch 3/5
10240/10240 [==============================] - 70s 7ms/step - loss: 1.6885 - categorical_accuracy: 0.2624 - val_loss: 1.7179 - val_categorical_accuracy: 0.2111
Epoch 4/5
10240/10240 [==============================] - 69s 7ms/step - loss: 1.6235 - categorical_accuracy: 0.2958 - val_loss: 1.7320 - val_categorical_accuracy: 0.2165
Epoch 5/5
10240/10240 [==============================] - 68s 7ms/step - loss: 1.5556 - categorical_accuracy: 0.3150 - val_loss: 1.7752 - val_categorical_accuracy: 0.2344



  3%|▎         | 1/36 [05:42<3:20:04, 342.99s/it]

Train on 10240 samples, validate on 1284 samples
Epoch 1/5
10240/10240 [==============================] - 70s 7ms/step - loss: 1.7863 - categorical_accuracy: 0.1986 - val_loss: 1.7465 - val_categorical_accuracy: 0.2344
Epoch 2/5
10240/10240 [==============================] - 67s 7ms/step - loss: 1.7394 - categorical_accuracy: 0.2247 - val_loss: 1.7272 - val_categorical_accuracy: 0.2484
Epoch 3/5
10240/10240 [==============================] - 66s 6ms/step - loss: 1.6946 - categorical_accuracy: 0.2491 - val_loss: 1.7061 - val_categorical_accuracy: 0.2578
Epoch 4/5
10240/10240 [==============================] - 66s 6ms/step - loss: 1.6283 - categorical_accuracy: 0.2740 - val_loss: 1.7197 - val_categorical_accuracy: 0.2586
Epoch 5/5
10240/10240 [==============================] - 66s 6ms/step - loss: 1.5592 - categorical_accuracy: 0.3092 - val_loss: 1.7447 - val_categorical_accuracy: 0.2453



  6%|▌         | 2/36 [11:22<3:13:42, 341.85s/it]

...


 94%|█████████▍| 34/36 [3:04:38<04:31, 135.57s/it]

Train on 10240 samples, validate on 1284 samples
Epoch 1/20
10240/10240 [==============================] - 12s 1ms/step - loss: 1.7984 - categorical_accuracy: 0.1978 - val_loss: 1.7507 - val_categorical_accuracy: 0.2375
Epoch 2/20
10240/10240 [==============================] - 11s 1ms/step - loss: 1.7482 - categorical_accuracy: 0.2249 - val_loss: 1.7312 - val_categorical_accuracy: 0.2469
Epoch 3/20
10240/10240 [==============================] - 10s 992us/step - loss: 1.7066 - categorical_accuracy: 0.2585 - val_loss: 1.6974 - val_categorical_accuracy: 0.2671
Epoch 4/20
10240/10240 [==============================] - 10s 983us/step - loss: 1.6414 - categorical_accuracy: 0.2990 - val_loss: 1.7117 - val_categorical_accuracy: 0.2562
Epoch 5/20
10240/10240 [==============================] - 10s 975us/step - loss: 1.5625 - categorical_accuracy: 0.3343 - val_loss: 1.7310 - val_categorical_accuracy: 0.2632
Epoch 6/20
10240/10240 [==============================] - 10s 975us/step - loss: 1.5056 - 


 97%|█████████▋| 35/36 [3:08:05<02:36, 156.91s/it]

Train on 10240 samples, validate on 1284 samples
Epoch 1/20
10240/10240 [==============================] - 14s 1ms/step - loss: 1.8033 - categorical_accuracy: 0.1998 - val_loss: 1.7511 - val_categorical_accuracy: 0.2150
Epoch 2/20
10240/10240 [==============================] - 12s 1ms/step - loss: 1.7318 - categorical_accuracy: 0.2339 - val_loss: 1.7192 - val_categorical_accuracy: 0.2539
Epoch 3/20
10240/10240 [==============================] - 12s 1ms/step - loss: 1.6967 - categorical_accuracy: 0.2584 - val_loss: 1.6953 - val_categorical_accuracy: 0.2710
Epoch 4/20
10240/10240 [==============================] - 12s 1ms/step - loss: 1.6330 - categorical_accuracy: 0.2890 - val_loss: 1.6839 - val_categorical_accuracy: 0.2656
Epoch 5/20
10240/10240 [==============================] - 12s 1ms/step - loss: 1.5609 - categorical_accuracy: 0.3326 - val_loss: 1.7095 - val_categorical_accuracy: 0.2780
Epoch 6/20
10240/10240 [==============================] - 12s 1ms/step - loss: 1.5074 - categori


100%|██████████| 36/36 [3:12:01<00:00, 320.03s/it]


In [0]:
from talos import Reporting
r = Reporting('051320043711.csv')
r.data

,round_epochs,val_loss,val_categorical_accuracy,loss,categorical_accuracy,batch_size,epochs,n_filters
0,5,1.775169,0.234424,1.555639,0.315039,10,5,32
1,5,1.744743,0.245327,1.559238,0.309180,10,5,64
2,5,1.726437,0.231308,1.597748,0.308398,10,5,128
3,10,2.068363,0.228972,1.362049,0.380273,10,10,32
4,10,1.810726,0.249221,1.422801,0.366309,10,10,64
5,10,1.872692,0.246106,1.392654,0.382715,10,10,128
6,20,2.943990,0.198598,1.070493,0.562695,10,20,32
7,20,2.392147,0.213396,1.141115,0.510254,10,20,64
8,20,2.153073,0.249221,1.128985,0.526855,10,20,128
9,5,1.719903,0.261682,1.568136,0.323145,50,5,32


In [0]:
from talos import  Predict
#from talos import best_model
r.high('val_categorical_accuracy')
#r.best_params(metric='val_categorical_accuracy', exclude = [])[0]


0.2694703936576843

## Hybrid CNN with History

In [0]:
def get_history_columns(keys, history_cols=['barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c', 'pants_on_fire_c']):
    d = {'pants_on_fire_c':0, 'false_c':1, 'barely_true_c':2, 'half_true_c':3, 'mostly_true_c':4, 'true_c':5}
    h_columns = [0, 0, 0, 0, 0]
    for i, h_c in enumerate(keys):
        if h_c in history_cols:
            h_columns[d[h_c]] = i
#             h_columns.append(i)
    return h_columns

def df_hist(df):
#     get_history_columns(df.keys())
    return np.array(df.iloc[:,get_history_columns(df.keys())]) - np.array(one_hot(np.array(df.label), 5), dtype=int)

def hist_account_for_current(a, df):
    print(df.label)
    return (a - one_hot(np.array(df.label, 5)))

In [0]:
def df_to_meta_input_with_H(df, tokenizers, columns):
    a = np.zeros((df.shape[0], len(columns)), dtype=int)
    for i, col in enumerate(columns):
        a[:,i] = np.array(tokenizers[i].texts_to_sequences(df[col])).reshape(-1)
    a = np.hstack((a, df_hist(df)))
    return a

In [0]:
X_h_train = df_hist(df_train)
X_h_val = df_hist(df_val)
X_h_test = df_hist(df_test)

# With sentiment
X_h_train = np.hstack((df_hist(df_train), train_senti.reshape(-1,1)))
X_h_val = np.hstack((df_hist(df_val), val_senti.reshape(-1,1)))
X_h_test = np.hstack((df_hist(df_test), test_senti.reshape(-1,1)))

# without

#X_h_train = df_hist(df_train)
#X_h_val = df_hist(df_val)
#X_h_test = df_hist(df_test)

In [0]:
# Liar
sentence_max_len = X_train.shape[1]
filter_sizes = (2,3,4) #
n_filters = 128 #
vocab_size = len(t.word_index) + 1
dimensions = 300
we_input_shape = (sentence_max_len,)
n_categories = len(l2i)
batch_size = 64 #
epochs = 5 #
dropout_prob = 0.8 #

number_of_meta_features = X_meta_train.shape[1]
md_input_shape = (number_of_meta_features,)
md_dimensions = dimensions #
# number_of_meta_features = 10
md_embedding_input_size = 2 #
md_filter_sizes = (3,6) # max is the amount of features can also have more than 2 e.g. (3, 4, 5)
md_n_filters = 10 #
lstm_size = 3 #
# max_pooling_pool_size = 1 #

history_size = (X_h_train.shape[1],)
h_out_dim = dimensions #


In [0]:
################################## Word Embeddings ~ we ##################################
# Input layer
we_input = Input(shape=we_input_shape, name='we_input')

# Embedding layer
we_embedding_layer = Embedding(vocab_size, dimensions, weights=[embedding_matrix], 
              input_length=sentence_max_len,trainable=False, name='we_embedding')(we_input)

convolutional_blocks = []
for i, filter_size in enumerate(filter_sizes):
    convolution_layer = Convolution1D(filters=n_filters, kernel_size=filter_size, 
                                      padding="valid", activation="relu", strides = 1,
                                      name='conv_layer' + str(i) + "_" + str(filter_size) + "_" 
                                      + str(n_filters))(we_embedding_layer)
    convolution_layer = GlobalMaxPool1D()(convolution_layer)
    convolutional_blocks.append(convolution_layer)
# print(we_input)
# if (len(filter_sizes)>1):
#     conv = Concatenate()(convolutional_blocks)
# else:
#     # untested
#     conv = convolution_layer

################################## Meta Data ##################################
# Input layer
# md_input = Input(shape=md_input_shape, name='md_input')

### Embedding layers ###
# md_input_split = Lambda(lambda x: tf.split(x, number_of_meta_features, axis=1))(md_input)
# print(md_input_split)
all_inputs = []
all_inputs.append(we_input)

md_input = Input(shape=md_input_shape, name="md_input")
all_inputs.append(md_input)
md_input_split = Lambda(lambda x: tf.split(x, number_of_meta_features, axis=1))(md_input)

history_input = Input(shape=history_size, name="history_input")
all_inputs.append(history_input)

dense_hist = Dense(h_out_dim, activation='tanh')(history_input)
reshape_dense_hist = Reshape((1, dense_hist.shape[1]))(dense_hist)
# print(dense_hist)
meta_embedding_layers = []
for i, c in enumerate(cols):
#     md_input = Input(shape=(1,))
    vocab_dim = len(ts[i].word_index) + 1
#     md_inputs.append(md_input)
    meta_embedding_layer = Embedding(vocab_dim, md_dimensions)(md_input_split[i])
    meta_embedding_layers.append(meta_embedding_layer)

meta_embedding_layers.append(reshape_dense_hist)
    
md_embed_concat = Concatenate(axis=1, name='md_embed_concat')(meta_embedding_layers)

md_convolutional_blocks = []
for i, filter_size in enumerate(md_filter_sizes):
    md_convolution_layer = Convolution1D(filters=md_n_filters, kernel_size=filter_size, 
                                      padding="valid", activation="relu", strides = 1,
                                      name='conv_layer' + str(i) + "_" + str(filter_size) + "_" 
                                      + str(n_filters))(md_embed_concat)

    md_convolution_layer = GlobalMaxPool1D()(md_convolution_layer)

    md_convolutional_blocks.append(md_convolution_layer)
if (len(filter_sizes)>1):
    md_conv = Concatenate()(md_convolutional_blocks)
else:
    # untested
    md_conv = m_d_convolution_layer

reshape_md_conv = Reshape((md_conv.shape[1], 1))(md_conv)

md_bilstm = Bidirectional(LSTM(lstm_size, return_sequences=False))(reshape_md_conv)

################################## Concat ##################################
convolutional_blocks.append(md_bilstm)

if (len(filter_sizes)>1):
    conv = Concatenate()(convolutional_blocks)
else:
    # untested
    conv = convolution_layer

################################## Output ##################################
dropout_layer = Dropout(dropout_prob, name="dropout")(conv)

dense_layer = Dense(n_filters, activation='relu', name='dense')(dropout_layer)
output_layer = Dense(n_categories, activation='softmax', name='output_dense')(dense_layer)

model = Model(all_inputs, output_layer)
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=["categorical_accuracy"])

In [0]:
model.fit([X_train, X_meta_train, X_h_train], Y_train, batch_size = batch_size, epochs=epochs, validation_data=([X_val, X_meta_val, X_h_val], Y_val))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 10240 samples, validate on 1284 samples
Epoch 1/5
10240/10240 [==============================] - 15s 1ms/step - loss: nan - categorical_accuracy: 0.1078 - val_loss: nan - val_categorical_accuracy: 0.0903
Epoch 2/5
10240/10240 [==============================] - 14s 1ms/step - loss: nan - categorical_accuracy: 0.0819 - val_loss: nan - val_categorical_accuracy: 0.0903
Epoch 3/5
10240/10240 [==============================] - 14s 1ms/step - loss: nan - categorical_accuracy: 0.0819 - val_loss: nan - val_categorical_accuracy: 0.0903
Epoch 4/5
10240/10240 [==============================] - 14s 1ms/step - loss: nan - categorical_accuracy: 0.0819 - val_loss: nan - val_categorical_accuracy: 0.0903
Epoch 5/5
10240/10240 [==============================] - 14s 1ms/step - loss: nan - categorical_accuracy: 0.0819 - val_loss: nan - val_categorical_accuracy: 0.0903


In [0]:
prediction = model.predict([X_test, X_meta_test, X_h_test], batch_size = batch_size)
Y_labels_classes = df_test.label.to_numpy()
np.sum(np.argmax(prediction, axis=1) == Y_labels_classes)/Y_labels_classes.shape[0]

0.07261247040252565

In [0]:
# look into history

#Tunining - Hybrid CNN with History

In [0]:
#pip install talos  #dependancy
import talos

sentence_max_len = X_train.shape[1]
#filter_sizes = (2,3,4) #
#n_filters = 128 #
vocab_size = len(t.word_index) + 1
#dimensions = 300
we_input_shape = (sentence_max_len,)
n_categories = len(l2i)
#batch_size = 64 #
#epochs = 5 #
#dropout_prob = 0.8 #

number_of_meta_features = X_meta_train.shape[1]
md_input_shape = (number_of_meta_features,)
md_dimensions = params['dimensions'] #
# number_of_meta_features = 10
md_embedding_input_size = 2 #
#md_filter_sizes = (3,6) # max is the amount of features can also have more than 2 e.g. (3, 4, 5)
#md_n_filters = 10 #
lstm_size = 3 #
# max_pooling_pool_size = 1 #

history_size = (X_h_train.shape[1],)
h_out_dim = params['dimensions'] #

def fakeNewsModel(x_train, y_train, x_val, y_val, params):
  we_input = Input(shape=we_input_shape, name='we_input')
# Embedding layer
  we_embedding_layer = Embedding(vocab_size, dimensions = params['dimensions'], weights=[embedding_matrix], 
                input_length=sentence_max_len,trainable=False, name='we_embedding')(we_input)

  convolutional_blocks = []
  for i, filter_size in enumerate(filter_sizes = params['filter_sizes']):
      convolution_layer = Convolution1D(filters=params['n_filters'], kernel_size=filter_size, 
                                      padding="valid", activation="relu", strides = 1,
                                      name='conv_layer' + str(i) + "_" + str(filter_size) + "_" 
                                      + str(params['n_filters']))(we_embedding_layer)
      convolution_layer = GlobalMaxPool1D()(convolution_layer)
      convolutional_blocks.append(convolution_layer)

  all_inputs = []
  all_inputs.append(we_input)

  md_input = Input(shape=md_input_shape, name="md_input")
  all_inputs.append(md_input)
  md_input_split = Lambda(lambda x: tf.split(x, number_of_meta_features, axis=1))(md_input)

  history_input = Input(shape=history_size, name="history_input")
  all_inputs.append(history_input)

  dense_hist = Dense(h_out_dim, activation='tanh')(history_input)
  reshape_dense_hist = Reshape((1, dense_hist.shape[1]))(dense_hist)
# print(dense_hist)
  meta_embedding_layers = []
  for i, c in enumerate(cols):
#     md_input = Input(shape=(1,))
      vocab_dim = len(ts[i].word_index) + 1
#     md_inputs.append(md_input)
      meta_embedding_layer = Embedding(vocab_dim, md_dimensions)(md_input_split[i])
      meta_embedding_layers.append(meta_embedding_layer)

  meta_embedding_layers.append(reshape_dense_hist)
    
  md_embed_concat = Concatenate(axis=1, name='md_embed_concat')(meta_embedding_layers)

  md_convolutional_blocks = []
  for i, filter_size in enumerate(params['md_filter_sizes']):
      md_convolution_layer = Convolution1D(filters= params['md_n_filters'], kernel_size=filter_size, 
                                      padding="valid", activation="relu", strides = 1,
                                      name='conv_layer' + str(i) + "_" + str(filter_size) + "_" 
                                      + str(n_filters))(md_embed_concat)

      md_convolution_layer = GlobalMaxPool1D()(md_convolution_layer)

      md_convolutional_blocks.append(md_convolution_layer)
  if (len(filter_sizes = params['filter_sizes'])>1):
      md_conv = Concatenate()(md_convolutional_blocks)
  else:
    # untested
      md_conv = m_d_convolution_layer

  reshape_md_conv = Reshape((md_conv.shape[1], 1))(md_conv)

  md_bilstm = Bidirectional(LSTM(lstm_size, return_sequences=False))(reshape_md_conv)

################################## Concat ##################################
  convolutional_blocks.append(md_bilstm)

  if (len(filter_sizes = params['filter_sizes'])>1):
      conv = Concatenate()(convolutional_blocks)
  else:
    # untested
      conv = convolution_layer

################################## Output ##################################
  dropout_layer = Dropout(dropout_prob = params['dropout_prob'], name="dropout")(conv)

  dense_layer = Dense(n_filters = paraams['n_filters'], activation='relu', name='dense')(dropout_layer)
  output_layer = Dense(n_categories, activation='softmax', name='output_dense')(dense_layer)

  model = Model(all_inputs, output_layer)
  model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=["categorical_accuracy"])
  out = model.fit([x_train[0], x_train[1], x_train[2]], y_train, batch_size = params ['batch_size'], epochs= params['epochs'], validation_data=([x_val[0], x_val[1], x_val[2]], y_val))

  return out, model

p = {'n_filters':[32, 64, 128],
     'filter_sizes' : [(2,3,4), (2,2,2), (3,3,3), (4,4,4), (5,5,5), (2,3,5)],
     'dimensions' : [100,200, 300],
     'batch_size': [50, 64, 100],
     'epochs' : [5,10,20],
     'dropout_prob' : [0.5, 0.8],
     'md_filter_sizes' : [(3,6), (3,4,5)],
     'md_n_filters' : [10, 15, 20]}

scan_object = talos.Scan(x=[X_train, X_meta_train, X_h_train],
                         y=Y_train,
                         x_val=[X_val, X_meta_val, X_h_val],
                         y_val=Y_val,
                         params=p,
                         model= fakeNewsModel,
                         experiment_name="test")

  0%|          | 0/3 [00:00<?, ?it/s]

Train on 10240 samples, validate on 1284 samples
Epoch 1/5
10240/10240 [==============================] - 13s 1ms/step - loss: nan - categorical_accuracy: 0.0887 - val_loss: nan - val_categorical_accuracy: 0.0903
Epoch 2/5
10240/10240 [==============================] - 12s 1ms/step - loss: nan - categorical_accuracy: 0.0819 - val_loss: nan - val_categorical_accuracy: 0.0903
Epoch 3/5
10240/10240 [==============================] - 12s 1ms/step - loss: nan - categorical_accuracy: 0.0819 - val_loss: nan - val_categorical_accuracy: 0.0903
Epoch 4/5
10240/10240 [==============================] - 12s 1ms/step - loss: nan - categorical_accuracy: 0.0819 - val_loss: nan - val_categorical_accuracy: 0.0903
Epoch 5/5
10240/10240 [==============================] - 12s 1ms/step - loss: nan - categorical_accuracy: 0.0819 - val_loss: nan - val_categorical_accuracy: 0.0903


 33%|███▎      | 1/3 [01:04<02:09, 64.90s/it]

Train on 10240 samples, validate on 1284 samples
Epoch 1/5
10240/10240 [==============================] - 14s 1ms/step - loss: nan - categorical_accuracy: 0.1182 - val_loss: nan - val_categorical_accuracy: 0.0903
Epoch 2/5
10240/10240 [==============================] - 12s 1ms/step - loss: nan - categorical_accuracy: 0.0819 - val_loss: nan - val_categorical_accuracy: 0.0903
Epoch 3/5
10240/10240 [==============================] - 12s 1ms/step - loss: nan - categorical_accuracy: 0.0819 - val_loss: nan - val_categorical_accuracy: 0.0903
Epoch 4/5
10240/10240 [==============================] - 12s 1ms/step - loss: nan - categorical_accuracy: 0.0819 - val_loss: nan - val_categorical_accuracy: 0.0903
Epoch 5/5
10240/10240 [==============================] - 12s 1ms/step - loss: nan - categorical_accuracy: 0.0819 - val_loss: nan - val_categorical_accuracy: 0.0903


 67%|██████▋   | 2/3 [02:11<01:05, 65.30s/it]

Train on 10240 samples, validate on 1284 samples
Epoch 1/5
10240/10240 [==============================] - 15s 1ms/step - loss: nan - categorical_accuracy: 0.1145 - val_loss: nan - val_categorical_accuracy: 0.0903
Epoch 2/5
10240/10240 [==============================] - 14s 1ms/step - loss: nan - categorical_accuracy: 0.0819 - val_loss: nan - val_categorical_accuracy: 0.0903
Epoch 3/5
10240/10240 [==============================] - 14s 1ms/step - loss: nan - categorical_accuracy: 0.0819 - val_loss: nan - val_categorical_accuracy: 0.0903
Epoch 4/5
10240/10240 [==============================] - 14s 1ms/step - loss: nan - categorical_accuracy: 0.0819 - val_loss: nan - val_categorical_accuracy: 0.0903
Epoch 5/5
10240/10240 [==============================] - 14s 1ms/step - loss: nan - categorical_accuracy: 0.0819 - val_loss: nan - val_categorical_accuracy: 0.0903


100%|██████████| 3/3 [03:26<00:00, 68.80s/it]
